### ***Network Attack Prediction Analysis***






**Purpose:** The purpose of this project is to test the capabilities of Generative AI models like OpenAI's GPT model in the automation of data analysis and generative predictions. In this example, we will be testing OpenAI's generative model in predicting network attacks using the [KDD Cup 1999 dataset](http://kdd.ics.uci.edu/databases/kddcup99/kddcup99.html).

**Methods (LLM Approach):** For this example, we will be loading the  KDD99 10% subset dataset into a pandas dataframe. We will then provide normalization/z scores (z=(x-mean)/std) to all of the numeric columns and provide One-Hot encoding/dummy variables to all the categorical columns. We will then utilize the LangChain framework in bridging our dataset with the OpenAI api for generative prediction.

#Project Notes


**Project To-Do's**



*   LangChain Local Data
*   Generative Model Prediction - Binary/ Multi-class
(Binary prediction first, Normal/ Malicious)
*   Smaller sample size 50k normal 10k malicious random
*   get rid of columns





**Project Notes**

*   Domain Knowledge - for DOS attack we know the protocol type will most likely ICMP
*   Features relavant to classes
normal - 1,6,12,15,16,17,18,19,31,32,37
smurf - 2,3,5,23,24,27,28,36,40,41
neptune - 4,25,26,29,30,33,34,35,38,39
land - 7
teardrop - 8
ftp_write - 9
back - 10,13
guess_pwd - 11
buffer_overflow - 14
warezclient - 22
*   batching and parallelisation for runtime
*   







**Project Questions/Problems**

*   How will the LLM infer something from continous numeric data?
*   Langchain is for human language data
*   It seems like we should take an embeddings approach using an embeddings model to calculate similarites
*   Or train a classification model






**Reads** [Similiar approach on dataset with LLM's](https://github.com/jeffheaton/t81_558_deep_learning/blob/master/t81_558_class_14_04_ids_kdd99.ipynb)


[Open Source Embeddings Model](https://huggingface.co/BAAI/bge-large-en-v1.5)

# Set up

Install libraries

In [ ]:
!pip install --upgrade openai
!pip install -U scikit-learn
!pip install pandas
!pip install pandasai
!python -m pip install -U pip
!python -m pip install -U matplotlib
!pip install langchain



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 1.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires tiktoken, which is not installed.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 31.9 MB/s eta 0:00:00
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.2.2
    Uninstalling scikit-learn-1.2.2:
      Successfully uninstalled scikit-learn-1.2.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.9/96.9 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 807.0/807.0 kB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 24.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 15.2 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 23.1.2
    Uninstalling pip-23.1.2:
      Successfully uninstalled pip-23.1.2
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 53.4 MB/s eta 0:00:00
  Attempting uninstall: matplotlib
    Found existing installation: matplotlib 3.7.1
    Uninstalling matplotlib-3.7.1:
      Successfully uninstalled matplotlib-3.7.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
lida 0.0.10 requires fastapi, which is not installed.
lida 0.0.10 requires kaleido, which is not installed.
lida 0.0.10 requires python-multipart, which is not installed.
lida 0.0.10 requires uvicorn, which is not installed.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.3/43.3 

In [ ]:
!pip install sentence_transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 2.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... canceled
ERROR: Operation cancelled by user
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/base_command.py", line 169, in exc_logging_wrapper
    status = run_func(*args)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/req_command.py", line 242, in wrapper
    return func(self, options, args)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/commands/install.py", line 377, in run
    requirement_set = resolver.resolve(
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/resolution/resolvelib/resolver.py", line 92, in resolve
    result = self._result = resolver.resolve(
  File "/usr/local/lib/python3.10/dist-packages/pip/_vendor/resolvelib/resolvers.py", line 546, in resolve
    state = resolution.resolve(requirements, max_rounds=max_rounds)
  File "/usr/local/lib/python3.10/dist

In [ ]:
!pip install tqdm

ERROR: Operation cancelled by user


Load our general OpenAI API key

In [ ]:
import openai
openai.api_key = ""

Import our training data CSV into a pandas dataframe

In [ ]:
from google.colab import drive
import pandas as pd

drive.mount('/content/gdrive')
df_train = pd.read_csv('gdrive/My Drive/Automation of Data Analytics/trainingdata.csv')
df_train.head(5)

#Analysis

See the distribution of attack labels in the dataset

In [ ]:
df_train.groupby('label')['label'].count()

Check for missings or null values in the dataset

In [ ]:
#get the count of null values for each column
null_counts = df_train.isnull().sum()
#display columns of null values
null_columns = df_train.columns[df_train.isnull().any()]
df_train_nulls = df_train[null_columns]
print(df_train_nulls)
#display rows of null values
df_train_null_rows = df_train[df_train.isnull().any(axis=1)]
print(df_train_null_rows)

# Preprocessing (Normalization)


Define the functions for calculating z scores

One Hot coding / dummy variables for categorical columns

In [ ]:
# Encode a numeric column as zscores
def encode_numeric_zscore(df, name, mean=None, sd=None):
    if mean is None:
        mean = df[name].mean()

    if sd is None:
        sd = df[name].std()

    df[name] = (df[name] - mean) / sd

# Encode text values to dummy variables(i.e. [1,0,0],[0,1,0],[0,0,1]
# for red,green,blue)
def encode_text_dummy(df, name):
    dummies = pd.get_dummies(df[name])
    for x in dummies.columns:
        dummy_name = f"{name}-{x}"
        df[dummy_name] = dummies[x]
    df.drop(name, axis=1, inplace=True)

Run z scores

In [ ]:
# Now encode the feature vector

pd.set_option('display.max_columns', 6)
pd.set_option('display.max_rows', 5)

for name in df_train.columns:
  if name == 'label':
    pass
  elif name in ['protocol_type','service','flag','land','logged_in',
                'is_host_login','is_guest_login']:
    encode_text_dummy(df_train,name)
  else:
    encode_numeric_zscore(df_train,name)

# display 5 rows

df_train.dropna(inplace=True,axis=1)
df_train[0:5]

<ipython-input-5-d8377fec7b18>:17: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[dummy_name] = dummies[x]
<ipython-input-5-d8377fec7b18>:17: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[dummy_name] = dummies[x]
<ipython-input-5-d8377fec7b18>:17: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df

,duration,src_bytes,dst_bytes,...,is_host_login-0,is_guest_login-0,is_guest_login-1
0,-0.067792,-0.002879,0.138664,...,1,1,0
1,-0.067792,-0.002820,-0.011578,...,1,1,0
2,-0.067792,-0.002824,0.014179,...,1,1,0
3,-0.067792,-0.002840,0.014179,...,1,1,0
4,-0.067792,-0.002842,0.035214,...,1,1,0


Change the label columns to binary normal / attack masks and split them into their own dataframes

In [ ]:
normal_mask = df_train['label']=='normal.'
attack_mask = df_train['label']!='normal.'

#if we want to drop the label column from the dataset
#df_train.drop('label',axis=1,inplace=True)

df_normal = df_train[normal_mask]
df_attack = df_train[attack_mask]

print(f"Normal count: {len(df_normal)}")
print(f"Attack count: {len(df_attack)}")

#Preprocessing (embeddings approach)


Label the categorical and numerical columns and split into label and target sets. Then list the column types

In [ ]:
#Label Columns
categorical_columns = ['protocol_type', 'service', 'flag']
numerical_columns = df_train.drop(columns=categorical_columns + ['label']).columns.tolist()
X = df_train.drop(columns=['label'])
y = df_train['label']

#List types
for col in categorical_columns:
  print(f"{col}: {df_train[col].unique()}")
for col in numerical_columns:
  print(f"{col}: {df_train[col].unique()}")

Split the data into training and validation sets

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

Create transformers for handling unknown categories

In [ ]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer

preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_columns),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_columns)
    ])
X_train_preprocessed = preprocessor.fit_transform(X_train)
X_val_preprocessed = preprocessor.transform(X_val)

Convert preprocessed data into textual format for LLM

In [ ]:
X_train_textual = [' '.join(map(str, row)) for row in X_train_preprocessed]
X_val_textual = [' '.join(map(str, row)) for row in X_val_preprocessed]

Diplay distribution of categorical columns

In [ ]:
#Visualize Distribution
import matplotlib.pyplot as plt
import seaborn as sns

for col in categorical_columns:
  plt.figure(figsize=(10,5))
  sns.countplot(data=df_train, x=col)
  plt.title(f"Distribution of {col}")
  plt.xticks(rotation=45)
  plt.show()


Display distribution of numerical columns (histograms)

In [ ]:
df_train[numerical_columns].hist(figsize=(15,15), bins=50)
plt.tight_layout()
plt.show()

Identify outliers of numerical columns with box plots

In [ ]:
for col in numerical_columns:
    plt.figure(figsize=(5,5))
    sns.boxplot(df_train[col])
    plt.title(f"Boxplot of {col}")
    plt.show()

Display correlation between numerical fields

In [ ]:
#correlation analysis
correlation_matrix = df_train[numerical_columns].corr()
plt.figure(figsize=(15,10))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm')
plt.title("Correlation Matrix of Numerical Fields")
plt.show()

#LangChain / HuggingFace Embeddings Approach

In [ ]:
# Encode text values to dummy variables(i.e. [1,0,0],[0,1,0],[0,0,1]
# for red,green,blue)
def encode_text_dummy(df, name):
    dummies = pd.get_dummies(df[name])
    for x in dummies.columns:
        dummy_name = f"{name}-{x}"
        df[dummy_name] = dummies[x]
    df.drop(name, axis=1, inplace=True)

In [ ]:
for name in df_train.columns:
  if name == 'label':
    pass
  elif name in ['protocol_type','service','flag','land','logged_in',
                'is_host_login','is_guest_login']:
    encode_text_dummy(df_train,name)

In [ ]:
X = df_train.drop(columns=['label'])
y = df_train['label']

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
X_train_textual = [' '.join(map(str, row)) for row in X_train.values]
X_val_textual = [' '.join(map(str, row)) for row in X_val.values]

Our LangChain imports

In [ ]:
from langchain.embeddings import HuggingFaceBgeEmbeddings

Set up model with Langchain

In [ ]:
embedding_model = HuggingFaceBgeEmbeddings(
    model_name="BAAI/bge-large-en-v1.5",
    model_kwargs={'device': 'cuda'},
    encode_kwargs={'normalize_embeddings': True}
)

In [ ]:
# @title Default title text
from multiprocessing import Pool, cpu_count
import multiprocessing
from tqdm import tqdm

def generate_batch_embeddings(batch):
    return [embedding_model.embed_documents(feature) for feature in batch]

def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]


num_cores = 2
# Reduce the batch size for more frequent updates
batch_size = len(X_train_textual) // (num_cores * 10)  # Adjust the multiplier as needed

X_train_batches = list(chunks(X_train_textual, batch_size))
X_val_batches = list(chunks(X_val_textual, batch_size))

multiprocessing.set_start_method('spawn', force=True)

with tqdm(total=len(X_train_batches), desc="Generating train embeddings") as pbar:
    with Pool(num_cores) as pool:
        for _ in pool.imap_unordered(generate_batch_embeddings, X_train_batches):
            pbar.update(1)
            train_embeddings.extend(_)

with tqdm(total=len(X_val_batches), desc="Generating validation embeddings") as pbar:
    with Pool(num_cores) as pool:
        for _ in pool.imap_unordered(generate_batch_embeddings, X_val_batches):
            pbar.update(1)
            val_embeddings.extend(_)

In [ ]:
from multiprocessing import Pool, cpu_count
import multiprocessing
from tqdm import tqdm

def generate_batch_embeddings(batch):
    return [embedding_model.embed_documents(feature) for feature in batch]
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

num_cores = 2
batch_size = len(X_train_textual) // num_cores

X_train_batches = list(chunks(X_train_textual, batch_size))
X_val_batches = list(chunks(X_val_textual, batch_size))
multiprocessing.set_start_method('spawn', force=True)
with Pool(num_cores) as pool:
    train_embeddings = list(tqdm(pool.imap(generate_batch_embeddings, X_train_batches), total=len(X_train_batches), desc="Generating train embeddings"))
    val_embeddings = list(tqdm(pool.imap(generate_batch_embeddings, X_val_batches), total=len(X_val_batches), desc="Generating validation embeddings"))
train_embeddings = [item for sublist in train_embeddings for item in sublist]
val_embeddings = [item for sublist in val_embeddings for item in sublist]

In [ ]:
from tqdm import tqdm

# Generate embeddings for training data using LangChain with progress bar
train_embeddings = []
for feature in tqdm(X_train_textual, desc="Generating train embeddings"):
    embedding = embedding_model.embed_documents(feature)
    train_embeddings.append(embedding)

# Generate embeddings for validation data using LangChain with progress bar
val_embeddings = []
for feature in tqdm(X_val_textual, desc="Generating validation embeddings"):
    embedding = embedding_model.embed_documents(feature)
    val_embeddings.append(embedding)

In [ ]:
print(df_train.shape)

(494021, 122)


In [ ]:
print(len(X_train))
print(len(X_val))

395216
98805


In [ ]:
print(len(X_train_textual))
print(len(X_val_textual))

121
121


In [ ]:
predictions_val = []
for val_emb in tqdm(val_embeddings, desc="Generating predictions"):
    similarities = cosine_similarity([val_emb], train_embeddings)
    most_similar_index = similarities.argmax()
    predictions_val.append(y_train.iloc[most_similar_index])

In [ ]:
print("Sample Predictions:")
for i in range(10):  # printing first 10 predictions
   print(f"Features: {X_val_textual[i]}")
   print(f"Predicted Attack Type: {predictions_val[i]}")
   print(f"Actual Attack Type: {y_val.iloc[i]}")
   print("------")

#OpenAI Embeddings Ada Model Approach

Generate embeddings for training and validation data (need loaded credits to run)

Problems: Run-time

In [ ]:
for feature in X_train_textual:
    embedding = openai.Embedding.create(input=feature, model="text-embedding-ada-002")

print(type(embedding))
print(embedding)

In [ ]:
train_embeddings = []
for feature in X_train_textual:
    embedding = openai.Embedding.create(input=feature, model="text-embedding-ada-002")
   # train_embeddings.append(embedding['data']['embedding'])
#val_embeddings = []
#for feature in X_val_textual:
   # embedding = openai.Embedding.create(input=feature, model="text-embedding-ada-002")
   # val_embeddings.append(embedding['data']['embedding'])

Generate predictions

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

predictions_val = []
for val_emb in val_embeddings:
        similarities = cosine_similarity([val_emb], train_embeddings)
        most_similar_index = similarities.argmax()
        predictions_val.append(y_train.iloc[most_similar_index])

Display Sample predictions

In [ ]:
print("Sample Predictions:")
for i in range(10):  # printing first 10 predictions
   print(f"Features: {X_val_textual[i]}")
   print(f"Predicted Attack Type: {predictions_val[i]}")
   print(f"Actual Attack Type: {y_val.iloc[i]}")
   print("------")

Calculate accuracy and print detailed classification report

In [ ]:
from sklearn.metrics import accuracy_score, classification_report

accuracy = accuracy_score(y_val, predictions_val)
print(f"Accuracy: {accuracy:.4f}")

report = classification_report(y_val, predictions_val)
print("\nClassification Report:")
print(report)